In [117]:
import os
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import (    Input, Dense, GlobalAveragePooling2D, GlobalAveragePooling1D, TimeDistributed, Conv2D, Flatten,
    Concatenate, BatchNormalization, Activation, Dropout, Softmax, Multiply,
    Lambda, Permute)
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import cv2

In [ ]:
# Constants
IMG_SHAPE = (224, 224, 3)
BATCH_SIZE = 2
DISH_CLEANED_CSV = r"C:\Users\Rafi Kyandra\VSCODE\Semester 5\Bangkit MSIB\CapstoneGithub_Monisick_ML\CapstoneGithub\metadata\dish_cleaned.csv"
FRAMES_DIR = r"C:\Users\Rafi Kyandra\VSCODE\Semester 5\Bangkit MSIB\CapstoneGithub_Monisick_ML\CapstoneGithub\images\frames"
OVERHEAD_DIR = r"C:\Users\Rafi Kyandra\VSCODE\Semester 5\Bangkit MSIB\CapstoneGithub_Monisick_ML\CapstoneGithub\images\overhead"

In [121]:
# Load dish metadata
dish_data = pd.read_csv(DISH_CLEANED_CSV, header=None)
dish_data.columns = ['dish_id', 'total_calories', 'total_mass', 'total_fat', 'total_carb', 'total_protein']

# Dish IDs
dish_ids = dish_data['dish_id'].unique()

# Dataset splitting
np.random.seed(42)
np.random.shuffle(dish_ids)
val_ids = dish_ids[:2]
test_ids = dish_ids[2:4]
train_ids = dish_ids[4:]

In [167]:
class MultiInputGenerator(Sequence):
    def __init__(self, dish_ids, batch_size, dish_data):
        self.dish_ids = dish_ids
        self.batch_size = batch_size
        self.dish_data = dish_data
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.dish_ids) / self.batch_size))

    def on_epoch_end(self):
        np.random.shuffle(self.dish_ids)

    def __getitem__(self, index):
        batch_dish_ids = self.dish_ids[index * self.batch_size:(index + 1) * self.batch_size]

        rgb_imgs, depth_raw_imgs, depth_color_imgs, frames = [], [], [], []
        mass_labels, macro_labels = [], []

        for dish_id in batch_dish_ids:
            dish_row = self.dish_data[self.dish_data['dish_id'] == dish_id].iloc[0]
            mass_labels.append(dish_row['total_mass'])
            macro_labels.append([ 
                dish_row['total_calories'],
                dish_row['total_fat'],
                dish_row['total_carb'],
                dish_row['total_protein']
            ])

            # Load overhead images
            rgb_imgs.append(preprocess_image(os.path.join(OVERHEAD_DIR, f"{dish_id}", "rgb.png")))
            depth_raw_imgs.append(preprocess_image(os.path.join(OVERHEAD_DIR, f"{dish_id}", "depth_raw.png")))
            depth_color_imgs.append(preprocess_image(os.path.join(OVERHEAD_DIR, f"{dish_id}", "depth_color.png")))

            # Load frames independently (not as sequences)
            frame_dir = os.path.join(FRAMES_DIR, f"{dish_id}", "frames_sampled30")
            frame_paths = sorted([os.path.join(frame_dir, f) for f in os.listdir(frame_dir) if f.endswith(".jpeg")])
            frames += [preprocess_image(f) for f in frame_paths]

        return (
            {
                "rgb_input": np.array(rgb_imgs),
                "depth_raw_input": np.array(depth_raw_imgs),
                "depth_color_input": np.array(depth_color_imgs),
                "frame_input": np.array(frames),  # Frames handled independently
            },
            {
                "mass_output": np.array(mass_labels),
                "macro_output": np.array(macro_labels),
            },
        )


In [155]:
def build_model(input_shape_rgb, input_shape_depth_raw, input_shape_depth_color, frame_input_shape):
    # Input layers
    rgb_input = layers.Input(shape=input_shape_rgb, name="rgb_input")
    depth_raw_input = layers.Input(shape=input_shape_depth_raw, name="depth_raw_input")
    depth_color_input = layers.Input(shape=input_shape_depth_color, name="depth_color_input")
    
    # Frame input
    frame_input = layers.Input(shape=frame_input_shape, name="frame_input")  # shape=(224, 224, 3)

    # Frame Processing with InceptionV3 (for ingredient classification)
    base_rgb = InceptionV3(weights=None, include_top=False, input_shape=frame_input_shape, name="inception_rgb")
    x_rgb = base_rgb(frame_input)
    x_rgb = layers.GlobalAveragePooling2D()(x_rgb)
    frame_output = layers.Dense(64, activation="relu")(x_rgb)  # Classification layer for frames

    # Depth images processing (similar to Midas architecture)
    # For RGB input
    x_rgb = layers.Conv2D(32, (3, 3), activation='relu')(rgb_input)
    x_rgb = layers.MaxPooling2D()(x_rgb)
    x_rgb = layers.Flatten()(x_rgb)
    
    # For raw depth input
    x_depth_raw = layers.Conv2D(32, (3, 3), activation='relu')(depth_raw_input)
    x_depth_raw = layers.MaxPooling2D()(x_depth_raw)
    x_depth_raw = layers.Flatten()(x_depth_raw)
    
    # For color depth input
    x_depth_color = layers.Conv2D(32, (3, 3), activation='relu')(depth_color_input)
    x_depth_color = layers.MaxPooling2D()(x_depth_color)
    x_depth_color = layers.Flatten()(x_depth_color)

    # Concatenate all features
    x = layers.concatenate([x_rgb, x_depth_raw, x_depth_color, frame_output], axis=-1)

    # Dense layers for final prediction
    x = layers.Dense(128, activation="relu")(x)

    # Output layers
    mass_output = layers.Dense(1, name="mass_output")(x)  # Predict total mass
    macro_output = layers.Dense(4, name="macro_output")(x)  # Predict macro nutrients (calories, fat, carbs, protein)

    # Define the model
    model = models.Model(inputs=[rgb_input, depth_raw_input, depth_color_input, frame_input],
                         outputs=[mass_output, macro_output])

    return model


# Build the model
model = build_model(input_shape_rgb, input_shape_depth_raw, input_shape_depth_color, frame_input_shape)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss={
        "mass_output": "mean_squared_error",  # MSE for total mass prediction
        "macro_output": "mean_squared_error",  # MSE for macro nutrient prediction
    },
    metrics=["mae"]  # Mean Absolute Error for evaluation
)

# Summary of the model
model.summary()



Model: "functional_45"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ rgb_input           │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depth_raw_input     │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depth_color_input   │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ frame_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3060         │ (None, 222, 222,  │        896 │ rgb_input[0][0]   │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3061         │ (None, 222, 222,  │        896 │ depth_raw_input[… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3062         │ (None, 222, 222,  │        896 │ depth_color_inpu… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ inception_rgb       │ (None, 5, 5,      │ 21,802,784 │ frame_input[0][0] │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_178   │ (None, 111, 111,  │          0 │ conv2d_3060[0][0] │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_179   │ (None, 111, 111,  │          0 │ conv2d_3061[0][0] │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_180   │ (None, 111, 111,  │          0 │ conv2d_3062[0][0] │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 2048)      │          0 │ inception_rgb[0]… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_24          │ (None, 394272)    │          0 │ max_pooling2d_17… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_25          │ (None, 394272)    │          0 │ max_pooling2d_17… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_26          │ (None, 394272)    │          0 │ max_pooling2d_18… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_81 (Dense)    │ (None, 64)        │    131,136 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_92      │ (None, 1182880)   │          0 │ flatten_24[0][0], │
│ (Concatenate)       │                   │            │ flatten_25[0][0]

 Total params: 173,346,021 (661.26 MB)

 Trainable params: 173,311,589 (661.13 MB)

 Non-trainable params: 34,432 (134.50 KB)

In [168]:
# Train, val, test data generators
train_gen = MultiInputGenerator(train_ids, BATCH_SIZE, dish_data)
val_gen = MultiInputGenerator(val_ids, BATCH_SIZE, dish_data)
test_gen = MultiInputGenerator(test_ids, BATCH_SIZE, dish_data)

In [169]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss={
        "mass_output": "mean_squared_error",  # MSE for total mass prediction
        "macro_output": "mean_squared_error",  # MSE for macro nutrient prediction
    },
    metrics={
        "mass_output": ["mae"],  # MAE for total mass prediction
        "macro_output": ["mae"],  # MAE for macro nutrient prediction
    }
)

# Train the model
history = model.fit(
    train_gen, 
    epochs=50,  # Adjust the number of epochs as necessary
    validation_data=val_gen,
    verbose=1
)

# After training, evaluate the model on the test data
test_loss, test_mae = model.evaluate(test_gen)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node functional_45_1/concatenate_92_1/concat defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\Rafi Kyandra\VSCODE\Semester 5\Bangkit MSIB\CapstoneGithub\CapstoneGithubVenv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "c:\Users\Rafi Kyandra\VSCODE\Semester 5\Bangkit MSIB\CapstoneGithub\CapstoneGithubVenv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\Rafi Kyandra\VSCODE\Semester 5\Bangkit MSIB\CapstoneGithub\CapstoneGithubVenv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Users\Rafi Kyandra\VSCODE\Semester 5\Bangkit MSIB\CapstoneGithub\CapstoneGithubVenv\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\Rafi Kyandra\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py

In [153]:
print("Shape of input1:", input1.shape)
print("Shape of input2:", input2.shape)


NameError: name 'input1' is not defined